In [101]:
import os
import numpy as np
import pandas as pd

In [102]:
#true
def save_pocket(save_name,res_list):
    with open(save_name,'w') as ff:
        for i in res_list:
            lines = str(i[0])+' '+ str(i[1])+' '+ str(i[2])+' '+ str(i[3])
            ff.write(lines + "\n")


In [103]:
def name_to_simple(name):
    names = {
        "GLY" : 'G',
        "ALA" : 'A',
        "VAL" : 'V',
        "LEU" : 'L',
        "ILE" : 'I',
        "PRO" : 'P',
        "PHE" : 'F',
        "TYR" : 'Y',
        "TRP" : 'W',
        "SER" : 'S',
        "THR" : 'T',
        "CYS" : 'C',
        "MET" : 'M',
        "ASN" : 'N',
        "GLN" : 'Q',
        "ASP" : 'D',
        "GLU" : 'E',
        "LYS" : 'K',
        "ARG" : 'R',
        "HIS" : 'H'}
    return names.get(name,None)

In [104]:
#多链存在链序号相同的问题
def get_res_from_pdb(chain):
    res_list =[]
    for resID in chain.chainIDandresID.unique():
        t= chain.chainIDandresID==resID
        resName = name_to_simple(chain[t].resName.unique()[0])
        x , y , z = [ round(x,3) for x in chain[t][['x','y','z']].astype('float64').apply(lambda x:x.mean())]
        res = [resName,x, y,z]
        res_list.append(res)
    return res_list

In [105]:
#true多肽是一个链，蛋白双连or单链是一个链
def get_resNum(anno_floder_path,pdb_file):
    metas =[]
    chain_1_id=[]
    with open(os.path.join(anno_floder_path,pdb_file)) as f:
        line = f.readline().strip()
        while line:
            if line[0:4] =="ATOM":
                metas.append([line[17:20],line[21:26],line[30:38],line[38:46],line[46:54]])
            line = f.readline().strip()
    a = np.array([np.array(x) for x in metas])

    try:
    #To DataFrame
        columns = ["resName","chainIDandresID","x","y","z"]
        data = pd.DataFrame(a,columns=columns)

        res_list = get_res_from_pdb(data)
    except:
        res_list=[]
    return res_list 

In [106]:
#true ，get dir and file
path = "./pocket6A"
for pdb_file in os.listdir(path):
    if pdb_file.endswith (".txt"):
    #if pdb_file == "3h11_C_B.txt":
        save_name='./pocket_acid6A/'+ os.path.splitext(pdb_file)[0] + '.txt'
        if os.path.exists(save_name):
            print('already exist')
        else:
            res_list=get_resNum(path,pdb_file)
            save_pocket(save_name,res_list)
       

In [107]:
##处理pepbdb
path = "./pocket_pepbdb6A"
for pdb_file in os.listdir(path):
    if pdb_file.endswith (".txt"):
        save_name='./pocket_acid6A/'+ os.path.splitext(pdb_file)[0] + '.txt'
#         if os.path.exists(save_name):
#             print('already exist')
#         else:
#             print(pdb_file)
        res_list=get_resNum(path,pdb_file)
        save_pocket(save_name,res_list)

In [15]:
##处理pepbdb，首先查看是否有同样的数据,实际无
pepbdb_name = os.listdir('./pocket_pepbdb6A/')
more_name = os.listdir('./pocket6A/')
for i in more_name:
    name = i[:-6] + ".txt"
    if name in pepbdb_name:
        print(name)

# Extract sequence,fasta form

In [108]:
import os
import numpy as np
import pandas as pd

In [109]:
def get_sequence(anno_floder_path,pdb_file):
    with open(os.path.join(anno_floder_path,pdb_file)) as f:
        lines= f.readline()
        line = lines.strip()
        resList=[]
        while line:
            resList.append(line[0])
            lines= f.readline()
            line = lines.strip()
        resStr = "".join(resList)
        return  resStr

In [110]:
clusters_path ="./pocket_acid6A"
with open('./pocket6A_seq.fasta','w') as ff:
    for pdb_file in os.listdir(clusters_path):
        resStr = get_sequence(clusters_path ,pdb_file)
        name='>'+pdb_file
        ff.write(name)
        ff.write("\n")
        ff.write(resStr)
        ff.write("\n")

In [40]:
#多肽
def name_to_simple(name):
    names = {
        "GLY" : 'G',
        "ALA" : 'A',
        "VAL" : 'V',
        "LEU" : 'L',
        "ILE" : 'I',
        "PRO" : 'P',
        "PHE" : 'F',
        "TYR" : 'Y',
        "TRP" : 'W',
        "SER" : 'S',
        "THR" : 'T',
        "CYS" : 'C',
        "MET" : 'M',
        "ASN" : 'N',
        "GLN" : 'Q',
        "ASP" : 'D',
        "GLU" : 'E',
        "LYS" : 'K',
        "ARG" : 'R',
        "HIS" : 'H'}
    return names.get(name,None)

def get_peptide_seq(chain,pdb_file):
    resList =[]
    for resID in chain.resID.unique():
        t= chain.resID==resID
        resName = name_to_simple(chain[t].resName.unique()[0])
        resList.append(resName)
    try:
        resStr = "".join(resList)
    except:
        #print(pdb_file)
        resStr = " "
    return  resStr

def get_peptide_resNum(anno_floder_path,pdb_file):
    metas =[]
    chain_1_id=[]
    with open(os.path.join(anno_floder_path,pdb_file)) as f:
        line = f.readline().strip()
        while line:
            if line[0:4] =="ATOM":
                metas.append([line[17:20],line[23:26],line[30:38],line[38:46],line[46:54]])
            line = f.readline().strip()
    a = np.array([np.array(x) for x in metas])


    columns = ["resName","resID","x","y","z"]
    data = pd.DataFrame(a,columns=columns)
    return get_peptide_seq(data,pdb_file)

In [41]:
clusters_path =["./peptide_pepbdb6A","./peptide6A"]
with open('./peptide6A_seq.fasta','w') as ff:
    for cluster in clusters_path:
        for pdb_file in os.listdir(cluster):
            resStr = get_peptide_resNum(cluster ,pdb_file)
            if resStr:
                name='>'+pdb_file
                ff.write(name)
                ff.write("\n")
                ff.write(resStr)
                ff.write("\n")

# Screen the suitable data for peptides 5-15 and pockets 10-50

In [111]:
import os
import numpy as np
import pandas as pd

In [113]:
#多肽多一点
pocket_name = []
peptide_name = []
with open('./peptide6A_seq.fasta','r') as f:
    while True:
        line_name = f.readline().strip()
        if not line_name:
            break
        line = f.readline().strip()
        seq_len =len(line)
        if seq_len>4 and seq_len<=15:
            peptide_name.append(line_name)
with open('./pocket6A_seq.fasta','r') as f:
    while True:
        line_name = f.readline().strip()
        if not line_name:
            break
        line = f.readline().strip()
        seq_len = len(line)
        if seq_len>9 and seq_len<=50:
            pocket_name.append(line_name)
            
peptide_new = []
pocket_new = []
for name_i in peptide_name:
    if name_i in pocket_name:
        peptide_new.append(name_i)
for name_j in pocket_name:
    if name_j in peptide_new:
        pocket_new.append(name_j)
print(len(pocket_new))
print(len(peptide_new))

9049
9049


In [114]:
with open('./peptide6A_seq5_15.fasta','w') as ff:
    with open('./peptide6A_seq.fasta','r') as f:
        while True:
            line_name = f.readline().strip()
            if not line_name:
                break
            sequence = f.readline().strip()
            if line_name in peptide_new:
                ff.write(line_name)
                ff.write("\n")
                ff.write(sequence)
                ff.write("\n")
with open('./pocket6A_seq10_50.fasta','w') as ff:
    with open('./pocket6A_seq.fasta','r') as f:
        while True:
            line_name = f.readline().strip()
            if not line_name:
                break
            sequence = f.readline().strip()
            if line_name in pocket_new:
                ff.write(line_name)
                ff.write("\n")
                ff.write(sequence)
                ff.write("\n")

# mmseq2 cluster command

In [ ]:
#mmseq2命令
#se3 
#id5_c7.tsv
mmseqs easy-cluster examples/pocket6A_seq10_50.fasta clusterRes tmp  --min-seq-id 0.5 --cluster-mode 2 --cov-mode 1 -c 0.7
mmseqs easy-cluster examples/peptide6A_seq5_15.fasta clusterRes tmp  --min-seq-id 0.5 --cluster-mode 2 --cov-mode 1 -c 0.7
#id5_c6.tsv
mmseqs easy-cluster examples/pocket6A_seq10_50.fasta clusterRes tmp  --min-seq-id 0.5 --cluster-mode 2 --cov-mode 1 -c 0.6
mmseqs easy-cluster examples/peptide6A_seq5_15.fasta clusterRes tmp  --min-seq-id 0.5 --cluster-mode 2 --cov-mode 1 -c 0.6

# Get protbert data (plus space)

In [1]:
import os
import numpy as np
import pandas as pd

In [116]:
with open('./peptide6A_seq5_15_protbert.fasta','w') as ff:
    with open('./peptide6A_seq5_15.fasta','r') as f:
        while True:
            line_name = f.readline().strip()
            if not line_name:
                break
            sequence = f.readline().strip()
            ff.write(line_name[1:])
            ff.write("\n")
            Str = " ".join(sequence)
            ff.write(Str)
            ff.write("\n")

In [2]:
with open('./pocket6A_seq10_50_protbert.fasta','w') as ff:
    with open('./pocket6A_seq10_50.fasta','r') as f:
        while True:
            line_name = f.readline().strip()
            if not line_name:
                break
            sequence = f.readline().strip()
            ff.write(line_name[1:])
            ff.write("\n")
            Str = " ".join(sequence)
            ff.write(Str)
            ff.write("\n")